In [1]:
data <- read.csv("selected.csv")

In [2]:
pc <- c(0,0,0,0,0,0)
for(i in 0:5) {
    pc[i+1] <- length(data[data[, "Labels"]==i, "Labels"])
}

In [3]:
full_text <- paste(data$Text, sep=" ", collapse=" ")
#tokens <- c()
#for(t in data$Text) {
#    tokens <- c(tokens, strsplit(t, " ")[[1]])
#}
#tokens <- unique(tokens)
tokens <- unique(strsplit(full_text, " ")[[1]])

In [4]:
## multinomial model
ptc <- data.frame(
    token = tokens, 
    ct0 = 0,
    ct1 = 0,
    ct2 = 0,
    ct3 = 0,
    ct4 = 0,
    ct5 = 0,
    #ctp0 = 0, 
    #ctp1 = 0, 
    #ctp2 = 0, 
    #ctp3 = 0, 
    #ctp4 = 0, 
    #ctp5 = 0, 
    pt0 = 0,
    pt1 = 0,
    pt2 = 0,
    pt3 = 0,
    pt4 = 0,
    pt5 = 0
)



for(i in 0:5) {
    class_text <- strsplit(paste(data[data[, "Labels"]==i, "Text"], sep=" ", collapse=" "), " ")[[1]]
    
    for(j in 1:length(tokens)) {
        ptc[j,paste("ct",i, sep="")] <- sum(class_text==ptc[j, "token"]) + 1
        #ptc[j,paste("ctp",i, sep="")] <- length(tokens) + length(class_text) - ptc[j,paste("ct",i, sep="")]
    }
    #ptc[,paste("pt",i, sep="")] <- ptc[,paste("ct",i, sep="")] / ptc[,paste("ctp",i, sep="")]
    ptc[,paste("pt",i, sep="")] <- ptc[,paste("ct",i, sep="")] / (length(class_text) + length(tokens))
}
ptc <- ptc[,c("token","pt0", "pt1", "pt2", "pt3", "pt4", "pt5")]

In [5]:
## bernoulli model
ptc_b <- data.frame(
    token = tokens, 
    ct0 = 1,
    ct1 = 1,
    ct2 = 1,
    ct3 = 1,
    ct4 = 1,
    ct5 = 1,
    #ctp0 = 0, 
    #ctp1 = 0, 
    #ctp2 = 0, 
    #ctp3 = 0, 
    #ctp4 = 0, 
    #ctp5 = 0, 
    pt0 = 0,
    pt1 = 0,
    pt2 = 0,
    pt3 = 0,
    pt4 = 0,
    pt5 = 0
)



for(i in 0:5) {
    count <- 0
    l <- length(data[data[, "Labels"]==i, "Text"])
    for(t in data[data[, "Labels"]==i, "Text"]) {
        count <- count+1
        if(count%%100 == 0){
            print(c(count, l))
            flush.console()
        }
        text <- strsplit(t, " ")[[1]]
        for(j in 1:length(tokens)) {
            ptc_b[j,paste("ct",i, sep="")] <- ptc_b[j,paste("ct",i, sep="")] + any(text==ptc_b[j, "token"])
        }
    }   
    #ptc_b[,paste("ctp",i, sep="")] <- length(tokens) + length(data[data[, "Labels"]==i, "Text"]) - ptc_b[,paste("ct",i, sep="")]
    #ptc_b[,paste("pt",i, sep="")] <- ptc_b[,paste("ct",i, sep="")] / ptc_b[,paste("ctp",i, sep="")]
    ptc_b[,paste("pt",i, sep="")] <- ptc_b[,paste("ct",i, sep="")] / (length(tokens) + l)
}
ptc_b<- ptc_b[,c("token","pt0", "pt1", "pt2", "pt3", "pt4", "pt5")]

[1]  100 1654
[1]  200 1654
[1]  300 1654
[1]  400 1654
[1]  500 1654
[1]  600 1654
[1]  700 1654
[1]  800 1654
[1]  900 1654
[1] 1000 1654
[1] 1100 1654
[1] 1200 1654
[1] 1300 1654
[1] 1400 1654
[1] 1500 1654
[1] 1600 1654
[1]  100 1995
[1]  200 1995
[1]  300 1995
[1]  400 1995
[1]  500 1995
[1]  600 1995
[1]  700 1995
[1]  800 1995
[1]  900 1995
[1] 1000 1995
[1] 1100 1995
[1] 1200 1995
[1] 1300 1995
[1] 1400 1995
[1] 1500 1995
[1] 1600 1995
[1] 1700 1995
[1] 1800 1995
[1] 1900 1995
[1]  100 2114


In [ ]:
#write.csv(ptc, "prob_token_by_class.csv")
#write.csv(ptc_b, "prob_token_by_class_b.csv")
# save for later

In [ ]:
right <- 0
right_b <- 0
for(k in 1:length(data[,"Labels"])) {
    vec <- strsplit(data[k, "Text"], " ")[[1]]
    probs <- log(pc)
    probs_b <- log(pc)
    for(h in vec) {
        for(i in 0:5) {
            #print(ptc[ptc[,"token"]==h ,paste("pt",i, sep="")])
            probs[i+1] <- probs[i+1] + log(ptc[ptc[,"token"]==h ,paste("pt",i, sep="")])
            probs_b[i+1] <- probs_b[i+1] + log(ptc_b[ptc_b[,"token"]==h ,paste("pt",i, sep="")])
        }
    }
    if( (0:5)[probs == max(probs)] == data[k, "Labels"]) {
        right <- right + 1
    }
    if( (0:5)[probs_b == max(probs_b)] == data[k, "Labels"]) {
        right_b <- right_b + 1
    }
}
print(100*right/length(data[,"Labels"]))
print(100*right_b/length(data[,"Labels"]))